In [ ]:
!pip install pandas-profiling==3.1.0

In [1]:
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.cluster import KMeans
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

In [2]:
np.random.seed = 3050

In [3]:
data_set = pd.read_csv('dataset_customer_churn.csv', sep='^')
data_set = data_set.dropna()

In [ ]:
profile = ProfileReport(data_set)
profile

In [4]:
data_set = data_set.drop(['CD_ASSOCIADO','CODIGO_BENEFICIARIO','REALIZOU_EXODONTIA_COBERTA','REALIZOU_ENDODONTIA_COBERTA', 'A006_REGISTRO_ANS','A006_NM_PLANO','CD_USUARIO','CLIENTE','FORMA_PGTO_MENSALIDADE','QTDE_ATO_N_COBERTO_EXECUTADO','QTDE_ATENDIMENTOS'], axis=1)

In [5]:
y = data_set['SITUACAO']
data_set = data_set.drop(['SITUACAO'], axis=1)

dici_trad = {
    'DESATIVADO' : 0,
    'ATIVO':1
}

y =  y.replace(dici_trad)
y = pd.Series(y)

In [6]:
dummies = pd.get_dummies(data_set[['SEXO','ESTADO_CIVIL','REALIZOU_PROCEDIMEN_ALTO_CUSTO','DIAS_ATE_REALIZAR_ALTO_CUSTO','PLANO','CODIGO_FORMA_PGTO_MENSALIDADE']])

In [7]:
num = data_set.drop(['SEXO','ESTADO_CIVIL','REALIZOU_PROCEDIMEN_ALTO_CUSTO','DIAS_ATE_REALIZAR_ALTO_CUSTO','PLANO','CODIGO_FORMA_PGTO_MENSALIDADE'], axis=1)

In [8]:
x = pd.concat([dummies, num], axis=1)

TESTE E TREINO

In [11]:
x = x
y = y

x_treino,x_teste,y_treino,y_teste = train_test_split(x, y, test_size=0.3, stratify=y)

VIEW FUNCTION

In [9]:
def view_score(y_teste, p) :
  baseline = np.ones(p.shape)

  ac = accuracy_score(y_teste,p)
  pc = precision_score(y_teste,p)
  f1 = f1_score(y_teste,p)

  print("Pelo Accuracy: ",ac * 100,"%")
  print("Pelo Precision: ",pc * 100,"%")
  print("Pelo F1: ",f1 * 100,"%")
  print()
  print("BASELINE")

  ac_b = accuracy_score(y_teste,baseline)
  pc_b = precision_score(y_teste,baseline)
  f1_b = f1_score(y_teste,baseline)

  print("Pelo Accuracy: ",ac_b * 100,"%")
  print("Pelo Precision: ",pc_b * 100,"%")
  print("Pelo F1: ",f1_b * 100,"%")

In [10]:
def cross_validation(x, y, modelo):
  cv = StratifiedKFold(n_splits=5, shuffle=True)

  results = cross_validate(modelo, x, y,cv = cv)
  results = pd.DataFrame(results)
  media = results.test_score.mean() * 100
  desvio = results.test_score.std()
  print("Média Teste: %.2f" %(media),"%")
  print("Entre : [%.2f, %.2f]" %((media - 2 * desvio), (media + 2 * desvio)),"%")

RANDOM FOREST

In [15]:
rf = RandomForestClassifier(max_depth=3, n_estimators=10)
rf.fit(x_treino, y_treino)

p = rf.predict(x_teste)

view_score(y_teste, p)

Pelo Accuracy:  84.03258867180405 %
Pelo Precision:  89.34655829544933 %
Pelo F1:  80.82666943607852 %

BASELINE
Pelo Accuracy:  45.61030244847478 %
Pelo Precision:  45.61030244847478 %
Pelo F1:  62.647081534102725 %


In [16]:
cross_validation(x, y,modelo=rf)

Média Teste: 82.84 %
Entre : [82.82, 82.87] %


LOGISTIC REGRESSION

In [23]:
lr = LogisticRegression()

lr.fit(x_treino,y_treino)

p2 = lr.predict(x_teste)

view_score(y_teste,p2)

C:\Users\fossg\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pelo Accuracy:  82.07623050777094 %
Pelo Precision:  80.9881514752575 %
Pelo F1:  80.14714334981034 %

BASELINE
Pelo Accuracy:  45.61030244847478 %
Pelo Precision:  45.61030244847478 %
Pelo F1:  62.647081534102725 %


In [22]:
cross_validation(x,y,modelo=lr)

Média Teste: 81.91 %
Entre : [81.91, 81.92] %


MULTILAYER

In [63]:
mlp = MLPClassifier()

mlp.fit(x_treino,y_treino)

p3 = mlp.predict(x_teste)

view_score(y_teste, p3)

Pelo Accuracy:  84.63368015014314 %
Pelo Precision:  85.61360627355128 %
Pelo F1:  82.55246437725252 %

BASELINE
Pelo Accuracy:  45.61030244847478 %
Pelo Precision:  45.61030244847478 %
Pelo F1:  62.647081534102725 %


CLUSTERING

In [17]:
data_final = pd.concat([x,y], axis=1)

values = Normalizer().fit_transform(data_final.values)

km = KMeans(n_clusters=5,n_init=10,max_iter=300)
km_pred = km.fit_predict(values)

clus = data_final

clus['CLUSTER'] = km.labels_
descp = clus.groupby('CLUSTER')['QTDE_DIAS_ATIVO','QTDE_ATO_COBERTO_EXECUTADO']
descp = pd.DataFrame(descp.mean())
descp['CLIENTES'] = clus['CLUSTER'].value_counts()

print(descp)

         QTDE_DIAS_ATIVO  QTDE_ATO_COBERTO_EXECUTADO  CLIENTES
CLUSTER                                                       
0             225.981770                    4.508428     87383
1              10.970534                    0.216186      9774
2            1083.989597                    7.574297    238866
3              41.820820                    0.957039     15130
4              97.297954                    2.268237     34257


C:\Users\fossg\AppData\Local\Temp\ipykernel_18952\2332075485.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  descp = clus.groupby('CLUSTER')['QTDE_DIAS_ATIVO','QTDE_ATO_COBERTO_EXECUTADO']
